# Wrap-up lab

This lab goes through some of the topics seen during this course.
More advanced topics are tackled in parts II and III.

## Part I. Convolutional neural nets

**Question 1.** Edit the code below to make sure CIFAR-10 image datasets are properly prepared to feed a 
convolutional neural net.
CIFAR-10 is an image classification dataset made of 32x32 color images.

In [1]:
import os

os.environ["KERAS_BACKEND"] = "torch"

In [ ]:
import keras_core as keras
from keras.datasets import cifar10
from keras.utils import to_categorical

def prepare_small_cifar10():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype(float)[::2]  # Subsampling the dataset: do not edit
    x_test = x_test.astype(float)[::2]  # Subsampling the dataset: do not edit
    
    x_train /= 255.
    x_test /= 255.
    
    y_train = to_categorical(y_train)[::2]  # Subsampling the dataset: do not edit
    y_test = to_categorical(y_test)[::2]  # Subsampling the dataset: do not edit
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = prepare_small_cifar10()
print(x_train.max(), x_train.min())

In [ ]:
import keras_core as keras
from keras.datasets import cifar10
from keras.utils import to_categorical

def prepare_small_cifar10():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype(float)[::2]  # Subsampling the dataset: do not edit
    x_test = x_test.astype(float)[::2]  # Subsampling the dataset: do not edit
    
    # TODO here
    
    y_train = to_categorical(y_train)[::2]  # Subsampling the dataset: do not edit
    y_test = to_categorical(y_test)[::2]  # Subsampling the dataset: do not edit
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = prepare_small_cifar10()
print(x_train.max(), x_train.min())

In [ ]:
import matplotlib.pyplot as plt

for idx in range(10):
    plt.subplot(2, 5, idx + 1)
    plt.imshow(x_train[idx])
    plt.axis("off")

**Question 2.** Implement the model below using `keras`. 
If some hyper-parameters are not specified, you are free to set them as you think is best.


![leNet model](https://raw.githubusercontent.com/rtavenar/deep-notebooks/main/assets/convnet_fig.png)

In [ ]:
from keras.models import Sequential
from keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense

model = Sequential([
    InputLayer((32, 32, 3)),
    Conv2D(filters=6, kernel_size=(5, 5), padding="valid", activation="relu"),
    MaxPool2D(pool_size=2),
    Conv2D(filters=32, kernel_size=(5, 5), padding="same", activation="relu"),
    MaxPool2D(pool_size=2),
    Flatten(),
    Dense(units=64, activation="relu"),
    Dense(units=10, activation="softmax")
])
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
h = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=100)

**Question 3.** We wonder whether the last hidden layer (made of 64 neurons) is really necessary.
Set up a way to properly compare the previous model with a model that would not have this layer.
What can you conclude?

In [ ]:
smaller_model = Sequential([
    InputLayer((32, 32, 3)),
    Conv2D(filters=6, kernel_size=(5, 5), padding="valid", activation="relu"),
    MaxPool2D(pool_size=2),
    Conv2D(filters=32, kernel_size=(5, 5), padding="same", activation="relu"),
    MaxPool2D(pool_size=2),
    Flatten(),
    Dense(units=10, activation="softmax")
])
smaller_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
smaller_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=100)

**Question 4.** You might have seen a slight overfitting with the model of question 2.
Set up a way to prevent this overfitting and see how it impacts the generalization performance of the model.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(h.history["val_loss"], label="Validation loss")
plt.plot(h.history["loss"], label="Training loss")
plt.legend();

In [ ]:
from keras.layers import Dropout
from keras.regularizers import l2

model = Sequential([
    InputLayer((32, 32, 3)),
    Conv2D(filters=6, kernel_size=(5, 5), padding="valid", activation="relu"),
    Dropout(rate=0.2),
    MaxPool2D(pool_size=2),
    Conv2D(filters=32, kernel_size=(5, 5), padding="same", activation="relu"),
    Dropout(rate=0.2),
    MaxPool2D(pool_size=2),
    Flatten(),
    Dense(units=64, activation="relu", kernel_regularizer=l2(1e-4)),
    Dropout(rate=0.2),
    Dense(units=10, activation="softmax")
])
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
h = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=100)

In [ ]:
plt.plot(h.history["val_loss"], label="Validation loss")
plt.plot(h.history["loss"], label="Training loss")
plt.legend();

**Question 5.** Use a ResNet50 model pre-trained on ImageNet to which you plug a single logistic 
regression layer and use it for this CIFAR-10 classification task.
What do you observe? Can you explain it?
What would you suggest?

In [ ]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

x_train_processed = preprocess_input(x_train * 255)
x_test_processed = preprocess_input(x_test * 255)

model = Sequential([
    InputLayer((32, 32, 3)),
    ResNet50(weights="imagenet", include_top=False, input_shape=(32, 32, 3)), 
    Flatten(),
    Dense(units=10, activation="softmax")
])
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
h = model.fit(x_train_processed, y_train, validation_data=(x_test_processed, y_test), epochs=10, batch_size=100)

## Part II. Multi-input / multi-output models

It sometimes happen that one wants to either feed a model with multiple inputs (_e.g._ an image and its 
caption) or predict several aspects of a sample, for which multiple outputs will be required.
In this context, `keras`' Sequential model cannot be used anymore, but other options exist, in what is 
called `keras`' functional API.

The basic idea behind this functional API is that the model will be defined by providing placeholders
for its inputs and outputs.
If the outputs are defined in such a way that they are computed from the inputs, the computation graph 
will be built automatically by `keras`.

For example, in the code below:

```python
from keras.models import Model
from keras.layers import Input, Dense

layer1 = Dense(units=20, activation="relu")
layer2 = Dense(units=30, activation="relu")
layer3 = Dense(units=1, activation="sigmoid")

inputs = Input(input_shape=(d, ))
hidden1 = layer1(inputs)
hidden2 = layer2(hidden1)
outputs = layer3(hidden2)

model = Model(inputs, outputs)
```

In this piece of code, we define the layers we need and then use them as if they were simple functions.
The model is finally defined as connecting the `inputs` to the `outputs`.
Here, if we had several inputs, we could have provided a list of them and, similarly, if we had
several outputs, they could be passed as a list.
Compilation and fit of such models are very similar to what we had with the `Sequential` class, except 
that:
* at fit time, the correct number of inputs / outputs should be passed and inputs (resp. outputs) should 
be gathered in lists just as what was done at the model definition step;
* at compile time, if several outputs are provided, several losses should be passed, as in the following example:

```python
losses = ["categorical_crossentropy", "binary_crossentropy"]
loss_weights = [1., 1.]
model.compile(optimizer="adam", loss=losses, loss_weights=loss_weights, metrics=["accuracy"])
```

**Question 6.** The code below loads a version of the MNIST dataset in which some digits are rotated. The goal is to both recognize the digit (information stored in `y_*`) and whether or not it is rotated 
(information stored in `y_rot_*`).
Implement a neural network that 2 blocks (convolution + pooling) that provide a joint latent 
representation for the image and for each output a single dense layer is plugged to this joint 
representation.

In [10]:
import numpy
from keras.datasets import mnist
from keras.utils import to_categorical
import tensorflow as tf
from tf.image import rot90

numpy.random.seed(0)

def batch_rotate(images, indices):
    images_with_channel = images[:, :, :, numpy.newaxis]
    images_with_channel[indices] = rot90(images_with_channel[indices])
    return images_with_channel / 255.

def prepare_mnist():
    step = 3
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    rotate_train = numpy.random.choice(2, size=x_train.shape[0]).astype(bool)
    rotate_test = numpy.random.choice(2, size=x_test.shape[0]).astype(bool)
    x_train = batch_rotate(x_train, rotate_train)[::step]
    x_test = batch_rotate(x_test, rotate_test)[::step]
    y_train = to_categorical(y_train)[::step]
    y_test = to_categorical(y_test)[::step]
    rotate_train = rotate_train.reshape((-1, 1))[::step]
    rotate_test = rotate_test.reshape((-1, 1))[::step]
    return x_train, x_test, y_train, y_test, rotate_train, rotate_test
  
x_train, x_test, y_train, y_test, y_rot_train, y_rot_test = prepare_mnist()

In [ ]:
for idx in range(10):
    plt.subplot(2, 5, idx + 1)
    plt.imshow(x_train[idx])
    plt.axis("off")

In [ ]:
from keras.layers import Input
from keras.models import Model

model_conv = Sequential([
    InputLayer((28, 28, 1)),
    Conv2D(10, kernel_size=5, activation="relu"),
    MaxPool2D(pool_size=2),
    Conv2D(50, kernel_size=5, activation="relu"),
    MaxPool2D(pool_size=2),
    Flatten()
])

classif_rot = Dense(1, activation="sigmoid")
classif_digit = Dense(10, activation="softmax")

inputs = Input(shape=(28, 28, 1))
repr_latente = model_conv(inputs)
output_rot = classif_rot(repr_latente)
output_digit = classif_digit(repr_latente)

model = Model(inputs, [output_rot, output_digit])

losses = ["binary_crossentropy", "categorical_crossentropy"]
loss_weights = [1., 1.]
model.compile(optimizer="adam", loss=losses, loss_weights=loss_weights, metrics=["accuracy"])
model.fit(x_train, [y_rot_train, y_train], validation_data=(x_test, [y_rot_test, y_test]), epochs=10, batch_size=100)


Use the code below to visualize your predictions.

In [ ]:
import matplotlib.pyplot as plt

def plot_rot_mnist_preds(img, pred_class, pred_rot):
    plt.figure()
    plt.imshow(img.reshape((28, 28)), cmap="gray")
    plt.title("Predicted as class %d (p=%.2f) and rotation=%.2f" % 
        (pred_class.argmax(), pred_class[pred_class.argmax()], pred_rot)
    )

preds_rot, preds_class = model(x_test)
list_idx = [0, 10, 20]
for idx in list_idx:
    plot_rot_mnist_preds(x_test[idx], preds_class[idx].numpy(), preds_rot[idx].numpy())


## Part III. Setting constraints on weights

It can happen that a model's parameters have some actual real-world meaning.
In this context, it could occur that you would want to set constraints on these weights (_e.g._ non-negativity, unit-norm, etc.).

To do so, `keras` layers accept a `kernel_constraint` parameter, as described 
[here](https://keras.io/api/layers/constraints/).

This parameter accepts either one of the provided constraints (`NonNeg`, `UnitNorm`, etc.) or a function
that takes a weight tensor as input and returns a transformed version that fulfills the constraint.

**Question 7.** Suppose we are given a dataset for which we know the outputs are generated as a
linear combination of the inputs in which the coefficients are positive and sum to one.
Implement a model that could estimate these weights from input-output data.



In [ ]:
def gen_data():
    n_samples = 1000
    d = 10
    X = numpy.random.randn(n_samples, d)
    secret_weights = numpy.random.uniform(size=d)
    secret_weights /= secret_weights.sum()

    print("Secret weights are:")
    print(secret_weights)

    y = X.dot(secret_weights.reshape((-1, 1))) + .1 * numpy.random.randn(n_samples, 1)
    return X, y

X_train, y_train = gen_data()

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
from keras.constraints import Constraint
from tf.nn import relu
from tf.math import reduce_sum


class PosSumTo1(Constraint):
  """Constrains weight tensors to be positive and of unit sum."""
  def __call__(self, w):
    w = relu(w)
    return w / reduce_sum(w)

model = Sequential([
  InputLayer((10, )),
  Dense(1, activation="linear", kernel_constraint=PosSumTo1(), use_bias=False)
])
model.compile(loss="mse", optimizer="adam")
model.fit(X_train, y_train, epochs=10, batch_size=50)

In [ ]:
from keras.constraints import Constraint
from tf.nn import relu
from tf.math import reduce_sum




In [ ]:
print(model.layers[0].weights)
print(reduce_sum(model.layers[0].weights[0]))